# Kanzus Pipeline Example

This notebook demonstrates how the Kanzus pipeline is used to perform on-demand distributed analysis.

In [2]:
import os
import sys
import json
import time
import numpy as np


from funcx.sdk.client import FuncXClient
from gladier.client import GladierClient as GladierBaseClient
from globus_automate_client import (create_flows_client, create_action_client, create_flows_client)

## Creating and using pipelines

Here we create a simple pipeline to move data and run an analysis function. The pipeline is just two steps but shows how Globus Automate and funcX can be used to create a reliable and secure distributed flow.

### Register a function to use

Start by defining a function and registering it with funcX. This function will be used within the example pipeline.

In [3]:
fxc = FuncXClient()

def file_size(data):
    """Return the size of a file"""
    import os
    return os.path.getsize(data['pathname'])

func_uuid = fxc.register_function(file_size)

Test the function works on some data

In [4]:
payload = {'pathname': '/etc/hostname'}
theta_ep = '6c4323f4-a062-4551-a883-146a352a43f5'
res = fxc.run(payload, endpoint_id=theta_ep, function_id=func_uuid)

NameError: name 'tutorial_endpoint' is not defined

In [ ]:
fxc.get_result(res)

### Define a flow for the function

Now define a flow to perform a Globus Transfer and then run the above function.

In [7]:
flow_definition = {
  "Comment": "An analysis flow",
  "StartAt": "Transfer",
  "States": {
    "Transfer": {
      "Comment": "Initial transfer",
      "Type": "Action",
      "ActionUrl": "https://actions.automate.globus.org/transfer/transfer",
      "Parameters": {
        "source_endpoint_id.$": "$.input.source_endpoint", 
        "destination_endpoint_id.$": "$.input.dest_endpoint",
        "transfer_items": [
          {
            "source_path.$": "$.input.source_path",
            "destination_path.$": "$.input.dest_path",
            "recursive": False
          }
        ]
      },
      "ResultPath": "$.Transfer1Result",
      "Next": "Analyze"
    },
    "Analyze": {
      "Comment": "Run a funcX function",
      "Type": "Action",
      "ActionUrl": "https://api.funcx.org/automate",
      "ActionScope": "https://auth.globus.org/scopes/facd7ccc-c5f4-42aa-916b-a0e270e2c2a9/automate2",
      "Parameters": {
          "tasks": [{
            "endpoint.$": "$.input.fx_ep",
            "func.$": "$.input.fx_id",
            "payload": {
                "pathname.$": "$.input.pathname"
            }
        }]
      },
      "ResultPath": "$.AnalyzeResult",
      "End": True
    }
  }
}

Register and run the flow

In [8]:
flows_client = create_flows_client()
flow = flows_client.deploy_flow(flow_definition, title="Stills process workflow")
flow_id = flow['id']
flow_scope = flow['globus_auth_scope']
print(f'Newly created flow with id: {flow_id}')

Newly created flow with id: a7be9a6c-ffca-48c2-a431-758346250440


In [10]:
src_ep = 'ddb59aef-6d04-11e5-ba46-22000b92c6ec' # EP1
dest_ep = 'ddb59af0-6d04-11e5-ba46-22000b92c6ec' # EP2
filename = 'test.txt'

flow_input = {
    "input": {
        "source_endpoint": src_ep,
        "source_path": f"/~/{filename}",
        "dest_endpoint": dest_ep,
        "dest_path": f"/~/{filename}",
        "result_path": f"/~/out_{filename}",
        "fx_id": func_uuid,
        "fx_ep": theta_ep,
        "pathname": '/etc/hostname'
    }
}

In [ ]:
flow_action = flows_client.run_flow(flow_id, flow_scope, flow_input)
print(flow_action)
flow_action_id = flow_action['action_id']
flow_status = flow_action['status']
print(f'Flow action started with id: {flow_action_id}')
while flow_status == 'ACTIVE':
    time.sleep(10)
    flow_action = flows_client.flow_action_status(flow_id, flow_scope, flow_action_id)
    flow_status = flow_action['status']
    print(f'Flow status: {flow_status}')

In [ ]:
flow_action['details']['output']['AnalyzeResult']

## Gladier Beam XY Search

Gladier removes a lot of this complexity by managing registering functions and flows when they change. Here we create a Gladier client and specify the tools that will be used in the flow.

In [11]:
from gladier_kanzus.flows.search_flow import flow_definition

class KanzusXYSearchClient(GladierBaseClient):
    client_id = 'e6c75d97-532a-4c88-b031-8584a319fa3e'

    gladier_tools = [
        'gladier_kanzus.tools.XYSearch',
        'gladier_kanzus.tools.CreatePhil',
        'gladier_kanzus.tools.DialsStills',
        'gladier_kanzus.tools.XYPlot',
        'gladier_kanzus.tools.SSXGatherData',
        'gladier_kanzus.tools.SSXPublish',
    ]
    flow_definition = flow_definition


search_client = KanzusXYSearchClient()

ModuleNotFoundError: No module named 'gladier_kanzus'

Register the stills function in a container

In [ ]:
from gladier_kanzus.tools.dials_stills import funcx_stills_process as stills_cont
container =  '/home/rvescovi/.funcx/containers/dials_v1.simg'
dials_cont_id = fxc.register_container(location=container, container_type='singularity')
stills_cont_fxid = fxc.register_function(stills_cont, container_uuid=dials_cont_id)

Define input to the flow. This describes the dataset that will be analyzed and the parameters for analysis.

In [5]:
conf = {'local_endpoint': '8f2f2eab-90d2-45ba-a771-b96e6d530cad',
        'queue_endpoint': '23519765-ef2e-4df2-b125-e99de9154611',
        }

data_dir = '/eagle/APSDataAnalysis/SSX/Demo/test'
proc_dir = f'{data_dir}/xy'
upload_dir = f'{data_dir}/test_images'

flow_input = {
    "input": {
        #Processing variables
        "proc_dir": proc_dir,
        "data_dir": data_dir,
        "upload_dir": upload_dir,

        #Dials specific variables.
        "input_files": "Test_33_{00000..00010}.cbf", 
        "input_range": "00000..00010",
        "nproc": 10,
        "beamx": "-214.400",
        "beamy": "218.200",

        # xy search parameters
        "step": "1",

        # funcX endpoints
        "funcx_local_ep": conf['local_endpoint'],
        "funcx_queue_ep": conf['queue_endpoint'],

        # container hack for stills
        "stills_cont_fxid": stills_cont_fxid,

        # publication
        "trigger_name": f"{data_dir}/Test_33_00001.cbf"
    }
}


In [12]:
flow_input['input']

{'source_endpoint': 'ddb59aef-6d04-11e5-ba46-22000b92c6ec',
 'source_path': '/~/test.txt',
 'dest_endpoint': 'ddb59af0-6d04-11e5-ba46-22000b92c6ec',
 'dest_path': '/~/test.txt',
 'result_path': '/~/out_test.txt',
 'fx_id': '69c6601c-860f-4614-af7f-a69cdca9b8a5',
 'fx_ep': '6c4323f4-a062-4551-a883-146a352a43f5',
 'pathname': '/etc/hostname'}

In [7]:
phils_flow = search_client.start_flow(flow_input=flow_input)

[DEBUG] gladier.client::get_funcx_function_ids() Checking functions for <gladier_kanzus.tools.XYSearch object at 0x7f6384699a00>
[DEBUG] gladier.client::get_funcx_function_ids() Checking functions for <gladier_kanzus.tools.CreatePhil object at 0x7f6384699190>
[DEBUG] gladier.client::get_funcx_function_ids() Checking functions for <gladier_kanzus.tools.DialsStills object at 0x7f6384699a90>
[DEBUG] gladier.client::get_funcx_function_ids() Checking functions for <gladier_kanzus.tools.XYPlot object at 0x7f6384699c70>
[DEBUG] gladier.client::get_funcx_function_ids() Checking functions for <gladier_kanzus.tools.SSXGatherData object at 0x7f63846991c0>
[DEBUG] gladier.client::get_funcx_function_ids() Checking functions for <gladier_kanzus.tools.SSXPublish object at 0x7f63846994c0>
[INFO] gladier.client::get_funcx_function_ids() Registering function ssx_publish_funcx_id
[DEBUG] gladier.config::save() Saved local gladier config to <_io.TextIOWrapper name='gladier.cfg' mode='w' encoding='UTF-8'>


Check the results:
https://petreldata.net/kanzus/projects/ssx/globus%253A%252F%252Fc7683485-3c3f-454a-94c0-74310c80b32a%252Fssx%252Ftest_images/

# Kanzus Pipeline

The full Kanzus pipeline is designed to be triggered as data are collected. It moves data to ALCF, performs analysis, analyzes the PRIME results, and publishes results to the portal.

# Virtual Beamline